<a href="https://colab.research.google.com/github/fillion1/afillion_225a/blob/main/bms225a_ds_workshop_3_student_notebook_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
# install scanpy with additional dependences
# leiden for clustering, harmony for batch correction and skmisc which is needed for HVG selection
!pip install --quiet 'scanpy[leiden,harmony,skmisc]'

The following cell downloads the original data and creates the data files `/content/blood_RNA_raw_counts.h5ad` and `/content/blood_RNA_raw_counts_SCG12.h5ad` that we will use for our analysis.

In [ ]:
# @title Download Data and create the **raw data** files that we will use for our analysis
################# THIS IS NOT PART OF THE WORKFLOW ##################
import os

if not os.path.exists('/content/blood_RNA.h5ad'):
  # downloading the data from the paper takes ~1min
  !wget -O blood_RNA.h5ad --header="User-Agent: Mozilla/5.0" "https://figshare.com/ndownloader/files/38849385?private_link=8aaa376913da23ae0def" > /dev/null 2>&1


if not os.path.exists('/content/blood_RNA_raw_counts.h5ad'):
  # -- create the raw data by removing all processed data information from the author's file
  import scanpy as sc
  # load the single cell RNA seq dataset from the h5ad file
  adata_processed = sc.read_h5ad('blood_RNA.h5ad') # AnnData object
  X_raw_counts = adata_processed.layers['counts']
  # the var dataframe
  raw_var_df = adata_processed.raw.var.copy()
  for i in raw_var_df.columns:
    del raw_var_df[i]
  raw_var_df['gene_name'] = raw_var_df.index
  # the obs dataframe
  raw_obs_df = adata_processed.obs[['LIBRARY',	'CoLabs_patient',	'condition']]
  raw_obs_df.columns = ['LIBRARY_ID', 'patient_id', 'condition']
  # create the anndata object
  adata = sc.AnnData(X = X_raw_counts, var=raw_var_df, obs=raw_obs_df)
  adata.write_h5ad('blood_RNA_raw_counts.h5ad')
  adata=adata[ adata.obs['LIBRARY_ID'] == 'XAUT1-POOL-SCG12']
  adata.write_h5ad('blood_RNA_raw_counts_SCG12.h5ad')
  del adata, adata_processed # delete the all data

# Single cell RNA seq analysis (single batch)
## Let's do a single batch first
The `'blood_RNA_raw_counts_SCG12.h5ad'` file only contains the single-cell data from a single sequencing batch (library):

LIBRARY_ID: `XAUT1-POOL-SCG12`.

In [4]:
import pandas as pd
import scanpy as sc

In [5]:
### Settings (optional) ###

# Show more columns in output
pd.set_option('display.max_columns', 50)
# Set parameters for the figures // sc.settings.set_figure_params?
sc.settings.set_figure_params(dpi=80,fontsize=16, facecolor="white",figsize=(8,6),color_map='viridis')
# Suppress all warnings -- not recommended when working on your research projects!
import warnings
warnings.filterwarnings("ignore")
# print out the package versions
print('scanpy version:', sc.__version__)
print('pandas version:', pd.__version__)

scanpy version: 1.11.4
pandas version: 2.2.2


## STEP 0: Load the data (raw counts)

In [93]:
###################################################### <- this will mark the functions that will change/update adata
# [🔺] load the AnnData object from the h5ad file (blood_RNA_raw_counts_SCG12)

adata = sc.read_h5ad('blood_RNA_raw_counts_SCG12.h5ad')





In [94]:
# display a summary of AnnData object
adata

AnnData object with n_obs × n_vars = 6410 × 22741
    obs: 'LIBRARY_ID', 'patient_id', 'condition'
    var: 'gene_name'

In [9]:
# [🔺] lets explore what is in adata. In the following two cells display the obs and var dataframes





In [95]:
# [🔺] obs

print(adata.obs)



                            LIBRARY_ID  patient_id condition
AAACCTGAGCTAAACA-1_2  XAUT1-POOL-SCG12   XAUT1-HS7      UCNB
AAACCTGAGTCAATAG-1_2  XAUT1-POOL-SCG12   XAUT1-HS2        HC
AAACCTGCAATGGAGC-1_2  XAUT1-POOL-SCG12   XAUT1-HS4        HC
AAACCTGCAGTAAGAT-1_2  XAUT1-POOL-SCG12   XAUT1-HS4        HC
AAACCTGCATTAACCG-1_2  XAUT1-POOL-SCG12   XAUT1-HS3        HC
...                                ...         ...       ...
TTTGTCAGTTGTACAC-1_2  XAUT1-POOL-SCG12   XAUT1-HS3        HC
TTTGTCATCCAAGCCG-1_2  XAUT1-POOL-SCG12  XAUT1-HS12       UCV
TTTGTCATCCACGTGG-1_2  XAUT1-POOL-SCG12   XAUT1-HS6      UCNB
TTTGTCATCCGATATG-1_2  XAUT1-POOL-SCG12   XAUT1-HS6      UCNB
TTTGTCATCTACTCAT-1_2  XAUT1-POOL-SCG12   XAUT1-HS6      UCNB

[6410 rows x 3 columns]


In [96]:
# [🔺] var

print(adata.var)



                   gene_name
RP11-34P13.7    RP11-34P13.7
RP11-34P13.15  RP11-34P13.15
RP11-34P13.16  RP11-34P13.16
RP11-34P13.13  RP11-34P13.13
FO538757.2        FO538757.2
...                      ...
RSPH1                  RSPH1
IMMTP1                IMMTP1
LINC00315          LINC00315
AL133493.2        AL133493.2
FTCD                    FTCD

[22741 rows x 1 columns]


In [97]:
# adata.var_names stores the gene names. It is the same as the index of the var dataframe:
adata.var_names

Index(['RP11-34P13.7', 'RP11-34P13.15', 'RP11-34P13.16', 'RP11-34P13.13',
       'FO538757.2', 'FO538757.1', 'AP006222.2', 'RP4-669L17.10',
       'RP4-669L17.8', 'RP5-857K21.4',
       ...
       'AJ006998.2', 'UBE3AP2', 'AP000569.9', 'AP000320.6', 'AP001619.2',
       'RSPH1', 'IMMTP1', 'LINC00315', 'AL133493.2', 'FTCD'],
      dtype='object', length=22741)

In [98]:
# You can also directly access adata.X, which gives total elements and shape of dataframe
adata.X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5758880 stored elements and shape (6410, 22741)>

In [99]:
adata.obs_names

Index(['AAACCTGAGCTAAACA-1_2', 'AAACCTGAGTCAATAG-1_2', 'AAACCTGCAATGGAGC-1_2',
       'AAACCTGCAGTAAGAT-1_2', 'AAACCTGCATTAACCG-1_2', 'AAACCTGGTCGAATCT-1_2',
       'AAACCTGGTGAGTGAC-1_2', 'AAACCTGTCAGCACAT-1_2', 'AAACCTGTCGCCGTGA-1_2',
       'AAACCTGTCTCATTCA-1_2',
       ...
       'TTTGTCACACATCCAA-1_2', 'TTTGTCACAGAGTGTG-1_2', 'TTTGTCACAGGAACGT-1_2',
       'TTTGTCACATCATCCC-1_2', 'TTTGTCAGTCGGATCC-1_2', 'TTTGTCAGTTGTACAC-1_2',
       'TTTGTCATCCAAGCCG-1_2', 'TTTGTCATCCACGTGG-1_2', 'TTTGTCATCCGATATG-1_2',
       'TTTGTCATCTACTCAT-1_2'],
      dtype='object', length=6410)

## STEP 1: Quality control

In [100]:
# mitochondrial gene names start with "MT-" for human and "Mt-" for mouse
# adata.var_names calls the index column of the variables, .str opens string related methods
# .startswith("pattern") is a method to evaluate the beginning of a string based on a set pattern
adata.var_names.str.startswith("MT-")

array([False, False, False, ..., False, False, False])

In [101]:
######################################################
# [🔺] create a new column in adata.var dataframe that indicates if a gene is mitochondrial or not (True if it is / False otherwise)
# choose an informative name for the new column : )

adata.var['MTgenes'] = adata.var_names.str.startswith("MT-")

# adata.var = adata.var.drop('MTgens', axis = 1) #accediently added MTgens to table, used .drop to remove column





In [102]:
# verify that the column was added (you can just display the relevant dataframe)
adata.var.head()

,gene_name,MTgenes
RP11-34P13.7,RP11-34P13.7,False
RP11-34P13.15,RP11-34P13.15,False
RP11-34P13.16,RP11-34P13.16,False
RP11-34P13.13,RP11-34P13.13,False
FO538757.2,FO538757.2,False


In [103]:
# [🔺] pandas practice: select the var rows that correspond to mitochondrial genes.
# How many mitochondrial genes are in this dataset?
# Hint: adata.var is a dataframe and can be sliced as df[ condition based on a logical expression  ]
# Hint2: slice the var dataframe where the column you created above is equal to True

MTgenes = adata.var[adata.var['MTgenes'] == True]
# MTgenes is a new varible set equal to the dataframe being filtered by the contents of the bracket
# In order to filter the column of interest in the dataframe needs to be selected adata.var['MTgenes']
# The column selected then needs to be set to a comparison statement (==, >, <, etc.)
# All of this is then passed into the dataframe to filter and return only values that fit within filter
len(MTgenes)




28

In [104]:
# Alternative to code 34, this more explicity shows how the code is being processed
MTfilter = adata.var['MTgenes'] == True
MTgenes = adata.var[MTfilter]
len(MTgenes)

28

In [105]:
# [🔹] for practice you can also add a column that indicates whether a gene is ribosomal
# ribosomal genes start with "RP" for human, "Rp" for mouse

adata.var['RPgenes'] = adata.var_names.str.startswith("RP")

adata.var




,gene_name,MTgenes,RPgenes
RP11-34P13.7,RP11-34P13.7,False,True
RP11-34P13.15,RP11-34P13.15,False,True
RP11-34P13.16,RP11-34P13.16,False,True
RP11-34P13.13,RP11-34P13.13,False,True
FO538757.2,FO538757.2,False,False
...,...,...,...
RSPH1,RSPH1,False,False
IMMTP1,IMMTP1,False,False
LINC00315,LINC00315,False,False
AL133493.2,AL133493.2,False,False


In [106]:
RPfilter = adata.var['RPgenes'] == True
RPgenes = adata.var[RPfilter]
len(RPgenes)

4118

### Calculate QC metrics

In [107]:
adata # lets remind ourseleves about adata, the following command will add a lot of information

AnnData object with n_obs × n_vars = 6410 × 22741
    obs: 'LIBRARY_ID', 'patient_id', 'condition'
    var: 'gene_name', 'MTgenes', 'RPgenes'

In [108]:
######################################################
# [🔺]use the sc.pp.calculate_qc_metrics() function to calculate qc metrics
# replace '????' with the column that indicated mitochondrial genes so that we also compute “% of counts in genes marked as mitochondrial”
# inplace=True --> updates the adata dataframe in place (adding all the qc columns)
# log1p=True --> calculates log1p tranformed versions (e.g., mean of logarithmized counts)

# sc.pp - scanpy preprocessing methods directory
# .calculate_qc_metrics() is the function

sc.pp.calculate_qc_metrics(adata, qc_vars=['MTgenes'], inplace=True, log1p=True)

In [109]:
adata

AnnData object with n_obs × n_vars = 6410 × 22741
    obs: 'LIBRARY_ID', 'patient_id', 'condition', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MTgenes', 'log1p_total_counts_MTgenes', 'pct_counts_MTgenes'
    var: 'gene_name', 'MTgenes', 'RPgenes', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'

### Filter genes

In [110]:
######################################################
# [🔺]filter genes that are expressed in fewer than 5 cells
# .filter_genes takes first the data, then comparison values which can be min or max counts or cells

sc.pp.filter_genes(adata, min_cells = 5)







In [111]:
adata # notice that the number of genes decreased

AnnData object with n_obs × n_vars = 6410 × 17473
    obs: 'LIBRARY_ID', 'patient_id', 'condition', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MTgenes', 'log1p_total_counts_MTgenes', 'pct_counts_MTgenes'
    var: 'gene_name', 'MTgenes', 'RPgenes', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'

### Visualize QC metrics (cells)

In [112]:
# sc.pl - scanpy plotting methods directory .violin - violin plot taking data, followed by the parameters you want to plot as keys
# displays distribution of data in each key defined
# the three keys in this code are the common QC metrics used when evaluating data

sc.pl.violin(
    adata,
    keys = ["log1p_n_genes_by_counts", "log1p_total_counts", "pct_counts_MTgenes"],
    jitter=0.1,
    multi_panel=True,
    size=1
)

In [113]:
# .scatter generates a scatter plot using the data, followed by the x vs y parameters
# color is used to set a heat map varible to the displayed points

sc.pl.scatter(adata, "log1p_total_counts", "pct_counts_MTgenes", color="log1p_n_genes_by_counts",size=50)

### Filter cells

In [116]:
# [🔺] pandas practice: How many cells have log1p_total_counts above 8?
# first make a filter using comparison statements, you can run adata.obs and adata.var to find log1p_total_counts

Counts_gt_8 = adata.obs['log1p_total_counts'] > 8
len(adata.obs[Counts_gt_8])








1946

In [126]:
##################################################
# [🔺] filter out cells by choosing QC thresholds for 'log1p_total_counts' and 'pct_counts_MTgenes' based on the above plots
# ok to be lenient we can revisit this after clustering.
# Hint: AnnData can be sliced as a dataframe based on a condition (logical expression). E.g., df[df['age']>20]
# for AnnData the condition can be computed based on the obs dataframe: e.g., adata[ obs dataframe column is greater than 5 ]
# After slicing adata, don't forget to save the output back to adata: e.g. as we update a df variable df=df[df['age']>20].

Thres_total_counts = adata.obs.log1p_total_counts > 7.5
Thres_MT_counts = adata.obs.pct_counts_MTgenes < 10

len(adata.obs)








6410

In [ ]:
adata # notice that the number of cells decreased

### Doublet detection (scrublet)

- We expect very few doublets in this particular case. The majority of doublets were removed using genetic information / patient SNPs.


In [ ]:
#############################################
# doublet detection with scrublet
sc.pp.scrublet(adata)

In [ ]:
adata # notice the columns 'doublet_score' and 'predicted_doublet' were added in obs. Also .uns appeared.

In [ ]:
#### !!!
#### This is NOT part of the single cell analysis
#### we run this to create some (fake) detected doublets
#### because no doublets are actually detected in this sample
import numpy as np
adata.obs["predicted_doublet"] = np.random.rand(adata.n_obs) < 0.05

In [ ]:
# .value_counts(): count how many times a value appears in a column of a dataframe
adata.obs['predicted_doublet'].value_counts()

In [ ]:
##############################################
# [🔺] AnnData challenge: filter out detected doublets!
# Hint1: adata can be sliced as a DataFrame, i.e., adata[ some condition here ]
# Hint2: adata.obs has the predicted_doublet column which is False for the cells we want to keep











## STEP 2: Normalization and log-transformation

In [ ]:
##############################################
# Before normalization, we want to save the count data to a new layer. The .copy() in the end makes sure that the data in adata.X are duplicated!!
# Otherwise the adata.layers["counts"] and adata.X will be pointing to the same location in computer memory
# which means that when one of them changes the other one does too. We don't want that : )

adata.layers["counts"] = adata.X.copy()

In [ ]:
adata.layers["counts"].sum()

In [ ]:
adata

In [ ]:
##############################################
# [🔺] Normalize data to 10K total counts













- `target_sum = 10000` is kind of arbitrary but very widely used. If we don't specify the target_sum parameter, `sc.pp.normalize_total(adata)` by default will normalize to the median of the total counts.


In [ ]:
# The only thing that changed is the X matrix (gene expression),
# which is now normalized to sum to 10K
adata.X.sum(1)

In [ ]:
##############################################
# [🔺] Logarithmize the data











In [ ]:
adata.X.sum(1) # the X matrix changed. Each entry is now log(1+x)

In [ ]:
######################################################
# [🔺] save the log1p normalized counts to a new layer! Let's call it 'log1p'
# We want to keep these around (useful for plotting and differential expression analysis) and adata.X will change again later.











## STEP 3: Highly variable genes (feature selection)

In [ ]:
######################################################
sc.pp.highly_variable_genes(adata, n_top_genes = 2000,
                            flavor='seurat_v3',
                            layer='counts')

In [ ]:
# This added information in the var dataframe
adata.var

In [ ]:
# .sort_values(): sort a dataframe with respect to the values of a given column
# .head(10) prints the top 10
adata.var.sort_values('highly_variable_rank').head(10)

## STEP 4: Dimensionality reduction with PCA

In [ ]:
adata # lets remind ourseleves about adata, the PCA will add a more information

In [ ]:
#####################################################################
# Before PCA we typically scale features to unit variance (common step but not absolutely necessary)
# Note that this changes adata.X
sc.pp.scale(adata, zero_center=False) # we don't want to zero center.

In [ ]:
#####################################################################
# PCA: by default this function uses only the highly variable genes
sc.tl.pca(adata, n_comps=50) # specify number of PCs (default is 50)

In [ ]:
# notice that the .obsm appeared that holds representations of the cells (embeddings)
adata

In [ ]:
# X_pca is a matrix of reduced dimensions (n_comps=50 in our case).
adata.obsm['X_pca'].shape

### Visualize PCA

In [ ]:
sc.pl.pca(adata,size=50)

In [ ]:
sc.pl.pca(adata,color='patient_id',size=10)

In [ ]:
sc.pl.pca(adata,color=['CST3','CD79A'],size=50,layer='log1p')

## STEP 5: Nearest-neighbor graph, Clustering and UMAP projection

In [ ]:
##############################################################
# [🔺] construct a k nearest neighbors graph (knn)
# 1. choose which representation to use
# 2. choose n_neighbors. Also specify random_state for reproducibility.












In [ ]:
adata

In [ ]:
##############################################################
# community detection algorithm that is used to cluster cells
# based on the neighbors graph we computed above.
# --> the key_added parameter defines the name of the obs column that will be added
#     this column will have the computed cluster label for each cell
# --> the resolution parameter specifies how coarse or fine-grained clusters will be

sc.tl.leiden(adata, resolution=1, key_added='leiden') # default parameters

In [ ]:
adata

In [ ]:
##############################################################
# run umap on the neighbors graph we computed above
sc.tl.umap(adata, min_dist=0.5, spread=1.0) # default values for the two main parameters

In [ ]:
adata

### Visualize UMAP

In [ ]:
# the color parameter can be a list of genes. Gene expression is visualized from the specified layer
sc.pl.umap(adata,color=['CST3','CD79A'],size=50,layer='log1p')

In [ ]:
# the color parameter can also be categorical (e.g., below we specify the column from obs that holds the cluster labels we want)
sc.pl.umap(adata,color='leiden',size=50,legend_loc='on data')

In [ ]:
##############################################################
# [🔺] Cluster the cells at a lower resolution (e.g., resolution=0.5), and visualize again
# Important: Give a different name to the labels so that we don't overwrite the 'leiden' column in obs
# Hint: run the sc.tl.leiden function and then visualize with sc.pl.umap functions
# Note: you don't need to re-run the umap agorithm












In [ ]:
# [🔹] Experiment with different neighbors, leiden and umap parameters

# lets work on a copy of the AnnData object for this
bdata = adata.copy() # anything we do with bdata won't affect adata

### pp.neighbors, tl.leiden, tl.umap and plot with pl.umap













## Differential expression analysis

In [ ]:
##############################################################
sc.tl.rank_genes_groups(adata,
                        groupby='leiden_coarse',
                        method='wilcoxon',
                        layer='log1p')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata,
                                n_genes=6,
                                values_to_plot='logfoldchanges',
                                min_logfoldchange=3,
                                vmax=5,
                                vmin=-5,
                                cmap='bwr',
                                dot_max=0.7)


In [ ]:
##############################################################
# [🔺] annotate the clusters!!

clusters_to_labels = {
    '0':'?',
    '1':'??',
    '2':'???',
    '3':'????'

}

# use the above dict to create a new column in obs with all cells labeled
# this is done by first mapping the cluster column to the values of the dict
# and assigning the result to a new column in obs
adata.obs['???'] = adata.obs['???'].map(clusters_to_labels)


## umap plot with the new labels
sc.pl.umap(adata,color='???',size=50,legend_loc='on data',legend_fontoutline=2,legend_fontsize=50)

## STEP 6: Save the processed dataset

In [ ]:
adata.write_h5ad('blood_RNA_BMS225a_single_batch.h5ad') # this will be deleted when this session ends

In [ ]:
# [🔺] The h5ad file is too big to download so let's mount google drive from the Files tab and save it there
# adata.write_h5ad('/content/drive/MyDrive/your_project_folder/blood_RNA_BMS225a_single_batch.h5ad')

# Compact version of the analysis

In [ ]:
### [🔺] re-run all the commands that are necessary to get us from the raw data to the processed dataset
### For this compact version we can skip steps that print dataframes/ plot data / etc

new_data = sc.read_h5ad('blood_RNA_raw_counts_SCG12.h5ad')


#...


#...


#...


#...


#...


#...


#...


#...

### if everything is right we should be able to get the same umap as above
sc.pl.umap(new_data,...)

# DONE